# Evidenz zur Klassifikation von Wohnungsbewohnerkategorien (Datensatz "wohnungen_2.csv")
Dieses Notebook stellt ein Modell auf Basis der Evidenztheorie vor, das verwendet wird, um die Bewohnerkategorie einer Wohnung basierend auf den Merkmalen der Wohnung zu klassifizieren.

## Ziel
Das Ziel dieses Projekts ist es, die Zusammenhänge zwischen den Eigenschaften einer Wohnung und den Bewohnerkategorien zu modellieren. Wir verwenden die Evidenztheorie, um Wahrscheinlichkeiten vorherzusagen, auch wenn einige Informationen unvollständig sind.

## Übersicht
- **Datensatzbeschreibung**: Einführung in die Datenstruktur und Herausforderungen (z. B. unausgeglichene Klassen).
- **Vorverarbeitung**: Bereinigung und Transformation der Daten.
- **Modellaufbau**: Definition der Evidenztheorie und Anwendung.
- **Evaluierung**: Beispiele und Ergebnisse.

---

## Vorbereitung
Zuerst werden die benötigten Bibliotheken importiert.

In [1]:
import pandas as pd
from data_cleaning.text_to_numeric import get_zimmer, get_stockwerk, get_kindergarten, get_schule, get_bahn, get_miete, get_nebenkosten, get_alter, get_stadtmitte, get_kaution, get_kueche, get_bad, get_mobliert, get_quadratmeter
from combined_mass.combined_mass import calculate_combined_mass

## Reinigung des Datensatzes
- **Bewohnerkategorie**: Der Datensatz enthält mehrere Spalten, mit anderen Bewohnergruppen als in `wohnungen_1.csv`. Und Zwar gibt es jetzt die Bewohnerkategorie "SingleHighIncome" anstatt "Alleinerziehende". Diese werden in einer neuen Spalte `Bewohnerkategorie` zusammengefasst.
- **Wohnungsattribute**: Der Datensatz enthält auch neue Attribute wie "Entfernung zur Stadtmitte", die bereinigt werden müssen. Ansonsten gibt es Attribute wie "Heizung" nicht mehr, die in `wohnungen_1.csv` vorhanden waren. Die anderen Attribute wurden jedoch beibehalten.
- **Datenbereinigung**: Um den Datensatz korrekt nutzen zu können, müssen wir den Fließtext durch Zahlen ersetzen, damit die KI mit diesen Daten arbeiten kann. Dafür werden "nein" und "ja" in numerische Werte (0 und 1) umgewandelt. Werte wo es mehr als ein Wert gibt wie bei der Miete, wurde die Mitte der beiden Werte genommen. Dies Funktioniert über unser Python Skript "data_cleaning.py". Es wurden in diesem Skript weitere Funktionen hinzugefügt, um neu dazu gekommene Attribute wie "Entfernung zur Stadtmitte" zu bereinigen.

In [2]:
file_path = '../../data/Wohnungen_2.csv'
data = pd.read_csv(file_path, sep=';')

bewohnerkategorie = ["Studierende", "Kleinfamilie", "DINK", "SingleHighIncome", "Expatriate", "Rentnerpaar"]
data['Bewohnerkategorie'] = data[bewohnerkategorie].apply(lambda row: 'Keine' if all(row == 'nein') else row.idxmax(), axis=1)

data = data[data['Bewohnerkategorie'] != 'Keine']

columns_to_replace = ["Hausmeister", "Garage", "Aufzug", "Balkon", "Terrasse", "Kehrwoche"]

pd.set_option('future.no_silent_downcasting', True)
data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})

data['Zimmerzahl'] = data['Zimmerzahl'].apply(get_zimmer)
data['Stockwerk'] = data['Stockwerk'].apply(get_stockwerk)
data['Kindergarten'] = data['Kindergarten'].apply(get_kindergarten)
data['Schule'] = data['Schule'].apply(get_schule)
data['S-Bahn'] = data['S-Bahn'].apply(get_bahn)
data['Miete'] = data['Miete'].apply(get_miete)
data['Nebenkosten'] = data['Nebenkosten'].apply(get_nebenkosten)
data['Alter'] = data['Alter'].apply(get_alter)
data['Entfernung zur Stadtmitte'] = data['Entfernung zur Stadtmitte'].apply(get_stadtmitte)
data['Kaution'] = data['Kaution'].apply(get_kaution)
data['Kueche'] = data['Kueche'].apply(get_kueche)
data['Bad'] = data['Bad'].apply(get_bad)
data['Moebliert'] = data['Moebliert'].apply(get_mobliert)
data['Quadratmeter'] = data['Quadratmeter'].apply(get_quadratmeter)

print(data.head())

    Zimmerzahl  Stockwerk Hausmeister  Kindergarten  Schule  S-Bahn Garage  \
3            0          3           1             2       0       0      0   
4            2          4           0             1       2       0      0   
6            2          1           0             2       2       3      0   
7            2          0           0             2       2       3      1   
16           0          4           0             0       2       2      1   

    Miete  Nebenkosten  Alter  ... Kehrwoche  Moebliert  Quadratmeter  \
3     276           50     63  ...         0          0            25   
4     626          226     18  ...         0          0            96   
6     626          226      6  ...         0          0            96   
7     726          226      0  ...         0          0           106   
16    276           75     26  ...         1          0            25   

    Studierende  Kleinfamilie  DINK SingleHighIncome Expatriate  Rentnerpaar  \
3           

## Anwendung der künstlichen Intelligenz

Jetzt bestimmen wir die Menge Omega mit den Attributen die zu bestimmen sind, also die Bewohnerkategorie.

In [3]:
omega = ["Studierende", "Kleinfamilie", "DINK", "SingleHighIncome", "Expatriate", "Rentnerpaar"]

Wie in `evidenztheorie_wohnungen_1.ipynb` verwenden wir die Funktion `calculate_combined_mass()` um daraus die Bewohnerkategorie die am wahrscheinlichsten ist rauszuziehen.

In [5]:
# Gegebene Attribut-Wert-Kombinationen\n",
attributes_values = {
    'Zimmerzahl': 0,
    'Stockwerk': 3,
    'Schule': get_schule("nah"),
}

# Berechne die kombinierte Evidenz
combined_mass = calculate_combined_mass(data, attributes_values, omega)

# Ausgabe der kombinierten Evidenz
print("Kombinierte Evidenz:", combined_mass)

# Wahrscheinlichste Hypothese basierend auf der kombinierten Evidenz
most_likely = combined_mass.max_bel()  # Maximale Glaubwürdigkeit

# Extrahiere den Namen der Bewohnerklasse aus dem frozenset
if isinstance(most_likely, frozenset):
    most_likely = next(iter(most_likely), None)  # Extrahiere das einzige Element oder None, falls leer

print("Am wahrscheinlichsten ist:", most_likely)

Basismaße für Zimmerzahl=0: {set():0.7837837837837838; {'Kleinfamilie'}:0.21621621621621623; {'Studierende'}:0.0; {'DINK'}:0.0; {'SingleHighIncome'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Basismaße für Stockwerk=3: {set():0.8378378378378378; {'Studierende'}:0.10810810810810811; {'Kleinfamilie'}:0.05405405405405406; {'DINK'}:0.0; {'SingleHighIncome'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Kombinierte Evidenz: {{'Kleinfamilie'}:1.0; {'Studierende'}:0.0; {'DINK'}:0.0; {'SingleHighIncome'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Basismaße für Schule=2: {{'Studierende'}:0.5675675675675675; set():0.33783783783783783; {'Kleinfamilie'}:0.0945945945945946; {'DINK'}:0.0; {'SingleHighIncome'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Kombinierte Evidenz: {{'Kleinfamilie'}:1.0; {'Studierende'}:0.0; {'DINK'}:0.0; {'SingleHighIncome'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Kombinierte Evidenz: {{'Kleinfamilie'}:1.0; {'Studierende'}:0.0; {'DINK'}:0.0; {'SingleHighIncome